In [144]:
import pandas as pd

In [145]:
from sklearn import datasets

In [146]:
import tensorflow as tf

In [147]:
import itertools

In [148]:
import os

In [149]:
import tempfile

In [150]:
COLUMNS = ["crim", "zn", "indus", "nox", "rm", "age",
           "dis", "tax", "ptratio", "medv"]

In [151]:
training_set = pd.read_csv('/Users/wisdom/Downloads/boston_train/boston_train.csv', skipinitialspace=True, skiprows=1, names=COLUMNS)

In [152]:
test_set = pd.read_csv('/Users/wisdom/Downloads/boston_train/boston_test.csv', skipinitialspace=True, skiprows=1, names=COLUMNS)

In [153]:
prediction_set = pd.read_csv('/Users/wisdom/Downloads/boston_train/boston_predict.csv', skipinitialspace=True, skiprows=1, names=COLUMNS)

In [154]:
FEATURES =["crim", "zn", "indus", "nox", "rm", "age","dis", "tax", "ptratio"]

In [155]:
LABEL="medv"

In [156]:
feature_cols = [tf.feature_column.numeric_column(k) for k in FEATURES]

In [157]:
# How to get feature cols for features that are variable
# CATEGORICAL_COLUMNS = ['sex', 'n_siblings_spouses', 'parch', 'class', 'deck',
#                        'embark_town', 'alone']
# NUMERIC_COLUMNS = ['age', 'fare']

# feature_columns = []
# for feature_name in CATEGORICAL_COLUMNS:
#   vocabulary = dftrain[feature_name].unique()
#   feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))

# for feature_name in NUMERIC_COLUMNS:
#   feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))

In [183]:
estimator = tf.estimator.LinearRegressor(feature_columns=feature_cols, model_dir="train")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'train', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fe36d9cc090>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [159]:
def get_input_fn(data_set, num_epochs=None, batch_size=128, shuffle=True, steps=1000):
    def input_function():
        ds = tf.data.Dataset.from_tensor_slices(({k: data_set[k].values for k in FEATURES}, data_set[LABEL]))
        if shuffle:
            ds = ds.shuffle(steps)
        ds = ds.batch(batch_size).repeat(num_epochs)
        return ds
    return input_function

In [160]:
train = estimator.train(input_fn=get_input_fn(training_set,                                       
                                           num_epochs=None,                                      
                                           batch_size = 128,                                      
                                           shuffle=False),                                      
                                           steps=1000)

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from train/model.ckpt-21000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 21000 into train/model.ckpt.
INFO:tensorflow:loss = 58.15824, step = 21000
INFO:tensorflow:global_step/sec: 209.155
INFO:tensorflow:loss = 58.155666, step = 21100 (0.4

In [161]:
ev = estimator.evaluate(    
          input_fn=get_input_fn(test_set,                          
          num_epochs=1,                          
          batch_size = 128,                          
          shuffle=False))

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-01-04T00:10:48Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from train/model.ckpt-22000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2021-01-04-00:10:49
INFO:tensorflow:Saving dict for global step 22000: average_loss = 19.710777, global_step = 22000, label/mean = 22.08, loss = 19.710

In [162]:
loss_score = ev["loss"]
print("Loss: {0:f}".format(loss_score))

Loss: 19.710777


In [163]:
training_set[LABEL].describe()

count    400.000000
mean      22.625500
std        9.572593
min        5.000000
25%       16.600000
50%       21.400000
75%       25.025000
max       50.000000
Name: medv, dtype: float64

In [164]:
y = estimator.predict(    
         input_fn=get_input_fn(prediction_set,                          
         num_epochs=1,                          
         batch_size = 128,                          
         shuffle=False))

In [165]:
predictions = list(p["predictions"] for p in itertools.islice(y, 6))
print("Predictions: {}".format(str(predictions)))

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from train/model.ckpt-22000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Predictions: [array([35.048214], dtype=float32), array([20.16722], dtype=float32), array([24.363743], dtype=float32), array([34.904106], dtype=float32), array([14.38385], dtype=float32), array([19.95739], dtype=float32)]


In [166]:
pred_dicts = list(y)
for pred in pred_dicts:
    print(pred["predictions"][0])

In [170]:
def predict(x):
    examples = []
    for index, row in x.iterrows():
        feature = {}
        for col, value in row.iteritems():
            feature[col] = tf.train.Feature(float_list=tf.train.FloatList(value=[value]))
        example = tf.train.Example(
            features=tf.train.Features(
                feature=feature
            )
        )
        examples.append(example.SerializeToString())
    return imported.signatures["predict"](
    examples=tf.constant(examples))

In [168]:
tmpdir = tempfile.mkdtemp()
estimator_base_path = os.path.join(tmpdir, 'from_estimator')
feature_spec = tf.feature_column.make_parse_example_spec(feature_cols);
export_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec);
estimator_path = estimator.export_saved_model(estimator_base_path, export_input_fn)
imported = tf.saved_model.load(estimator_path)


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: ['serving_default', 'regression']
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predict']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from train/model.ckpt-22000
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: /var/folders/6_/8_x7k80s52vcfppnmvb31wxr0000gn/T/tmpak2cdhix/from_estimator/temp-b'1609719050'/saved_model.pb


In [178]:
result = predict(prediction_set)

In [181]:
print(result["predictions"].numpy())

[[35.048214]
 [20.16722 ]
 [24.363743]
 [34.904106]
 [14.38385 ]
 [19.95739 ]]
